# Imports

In [1]:
import os
import sys
import glob
import datetime

import pandas as pd

In [2]:
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../../eventpredictor'))

from eventpredictor.config import RAW_DATA_DIR
from eventpredictor.data.download_data import FootballScraper

2024-05-31 13:32:40.645 | INFO     | config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor
2024-05-31 13:32:40.652 | INFO     | eventpredictor.config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor


# Downloading data

In [3]:
fs = FootballScraper(
    start_date=datetime.date(2015, 5, 31),
    end_date=datetime.date(2024, 1, 31),
    raw_data_dir=RAW_DATA_DIR
)

## Events

In [4]:
fs.download_events()
fs.convert_events()
fs.combine_events()

/Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor/eventpredictor/data/download_data.py:52: DtypeWarning: Columns (13,33,34,36,37,42,124,131,132,133,134,135,136,137,140,143,144,145,146,147,148,149,152,153,154,157,158,159,160,161,162,163,164,165,166,167,170,171,172,175,176,177,178,179,180,181,182,183) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_list.append(pd.read_csv(file))


## Odds

In [5]:
fs.download_odds()
fs.conert_odds()
fs.combine_odds()

## Filtering events

In [6]:
countries_to_remove = {
    'africa',
    'asia',
    'esoccer',
    'europe',
    'international',
    'north-central-america',
    'oceania',
    'simulated-reality-women',
    'south-africa',
    'south-america',
    'world'
}

status_to_keep = ['Ended']

fs.generate_download_file(
    countries_to_remove=countries_to_remove,
    status_to_keep=status_to_keep
)

File exists already.


## Statistics

In [7]:
fs.download_statistics()
fs.convert_statistics()
fs.combine_statistics()

## Incidents

In [8]:
fs.download_incidents()
fs.convert_incidents()
fs.combine_incidents()

## Lineups

In [9]:
fs.download_lineups()
fs.convert_lineups()
fs.combine_lineups()

# What has been downloaded?

In [10]:
print(f"Number of downloaded scheduled_events [json]: {len(glob.glob(os.path.join(fs.scheduled_events_json, '*')))}")
print(f"Number of downloaded scheduled_events [csv]: {len(glob.glob(os.path.join(fs.scheduled_events_csv, '*')))}")

print(f"Number of downloaded odds [json]: {len(glob.glob(os.path.join(fs.odds_json, '*')))}")
print(f"Number of downloaded odds [csv]: {len(glob.glob(os.path.join(fs.odds_csv, '*')))}")

print(f"Number of downloaded statistics [json]: {len(glob.glob(os.path.join(fs.statistics_json, '*')))}")
print(f"Number of downloaded statistics [csv]: {len(glob.glob(os.path.join(fs.statistics_csv, '*')))}")

print(f"Number of downloaded incidents [json]: {len(glob.glob(os.path.join(fs.incidents_json, '*')))}")
print(f"Number of downloaded incidents [csv]: {len(glob.glob(os.path.join(fs.incidents_csv, '*')))}")

print(f"Number of downloaded lineups [json]: {len(glob.glob(os.path.join(fs.lineups_json, '*')))}")
print(f"Number of downloaded lineups [csv]: {len(glob.glob(os.path.join(fs.lineups_csv, '*')))}")

Number of downloaded scheduled_events [json]: 3168
Number of downloaded scheduled_events [csv]: 3168
Number of downloaded odds [json]: 3168
Number of downloaded odds [csv]: 3168
Number of downloaded statistics [json]: 99771
Number of downloaded statistics [csv]: 99771
Number of downloaded incidents [json]: 170572
Number of downloaded incidents [csv]: 170572
Number of downloaded lineups [json]: 125401
Number of downloaded lineups [csv]: 73577
